In [104]:
import pymysql
import pandas as pd
import pickle
import csv

In [245]:
# Open AWS database connection
db = pymysql.connect("****","***","****","dbprojectAWS" )

# prepare a cursor object using cursor() method
cursor = db.cursor()

In [137]:
#3. Get Member-Topics data
sql="select m.member_id, t.topic_name from members m \
inner join members_topics t on t.member_id=m.member_id \
where m.state='NY';"
try:
   #Execute the SQL command
    cursor.execute(sql)
    
   #Fetch all the rows in a list of lists.
    MTs = cursor.fetchall()
except:
    print ("Error: unable to fetch data")

In [138]:
df2 = pd.DataFrame( [[ij for ij in i] for i in MTs])
df2.rename(columns={0: 'Member_ID', 1: 'Topic_Name'}, inplace=True);
df2['Dummy']=1
df2.head()

,Member_ID,Topic_Name,Dummy
0,6,photo,1
1,6,art,1
2,6,newtech,1
3,6,parents,1
4,6,technology,1


In [139]:
df=df2.pivot_table(index=['Member_ID'], 
            columns=['Topic_Name'],values='Dummy').fillna(0)
df.head()

Topic_Name,20s-30s-social,20s-social,2d-game-development,80s-dancing,90s-music,Euro-games,a-spiritual-path-to-higher-creativity,abstract-art,acting-and-writing,acting-class-for-industry-professionals,...,writing-is-your-business,writing-lyrics,writing-songs,writing-workshops,yoga-dance-a-unique-movement-experience,yoga-dance-classes-with-world-music-meditations,young-adult-literature,young-professional-networking,youth-sports,zoo-animal-welfare-education
Member_ID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
interest=df.ix[6,0:]
Frame= df.ix[6:,0:]

In [155]:
import sklearn
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5).fit(Frame)

In [228]:
Near=(nbrs.kneighbors([interest]))
Near

(array([[ 0.        ,  1.73205081,  1.73205081,  1.73205081,  1.73205081]]),
 array([[     0, 101400,  93460,  79027,  83947]], dtype=int64))

In [242]:
x=list(Near[1:])[0][0][1:5]

a=list()
for y in x:
    a.append(df.index.values[y])
a

[116012502, 103303202, 80530052, 88781412]

In [253]:
Suggest=list()
for z in a:
    sql="select g.group_name from members m inner join groups g on g.group_id=m.group_id where m.member_id="+ str(z)
    try:
       #Execute the SQL command
        cursor.execute(sql)

       #Fetch all the rows in a list of lists.
        MTs = cursor.fetchall()
        Suggest.append(MTs)
    except:
        print ("Error: unable to fetch data")
Suggest   

[(('Out Professionals',),),
 (('NY Bloggers Networking Meetup Group',),
  ('New York Entrepreneurs & Startup Network (NYESN)',)),
 (('Figure Drawing NYC--Summer Sketch Group and Draw-mania!',),
  ('NY Tech Mixer',),
  ('The New York Creative Group',)),
 (('The New York Python Meetup Group',),
  ('NYC on Rails',),
  ('Girl Develop It NYC',),
  ('Hack && Tell',),
  ('NYC Apps',),
  ('NYC Nodejs',),
  ('Google NYC Tech Talks',),
  ('NYC.rb',),
  ('Code Meet Print (NY)',),
  ('League of Women Coders',))]